In [1]:
import sympy

In [12]:
#Defino todos los coeficientes del sistema
#que conforman a la matriz A
a,b,c,d,e,f,g,h,i = sympy.symbols("a,b,c,d,e,f,g,h,i")
#La variable de laplace
s = sympy.symbols("s")
#Las 3 matrices que representan al sistema en forma 
#de control
A = sympy.Matrix([[a,b,c], [d,e,f], [g,h,i]])
B = sympy.Matrix([[0],[0],[1]])
# No se bien porque sympy arma a C como una matrix de 3x1
# Si la transpongo la convierte en una fila y queda bien representado
# El sistema
C = sympy.Matrix([1,0,0]).T

## Calculo de la funcion de transferencia.
Partiendo de la representacion de la forma de control, 
especificada arriba, vamos a proceder a calcular la funcion de 
transferencia basado en la forma matricial

```H(s) = C (sI - A)**-1 B + D```


In [14]:
# H(s) = C (sI - A)**-1 B + D
temp = s * sympy.eye(3) - A
inv = temp.inv()

In [17]:
temp = C * inv 
H = temp * B

print(H)

Matrix([[(b*f - c*e + c*s)/(-a*e*i + a*e*s + a*f*h + a*i*s - a*s**2 + b*d*i - b*d*s - b*f*g - c*d*h + c*e*g - c*g*s + e*i*s - e*s**2 - f*h*s - i*s**2 + s**3)]])


In [11]:
x = sympy.symbols("x")
x *x

(b*f - c*e + c*s)/(-a*e*i + a*e*s + a*f*h + a*i*s - a*s**2 + b*d*i - b*d*s - b*f*g - c*d*h + c*e*g - c*g*s + e*i*s - e*s**2 - f*h*s - i*s**2 + s**3)

x**2